# Breakfast at the Frat: A Time Series Analysis

Sales and promotion information on the top five products from each of the top three brands within four selected categories (mouthwash, pretzels, frozen pizza, and boxed cereal), gathered from a sample of stores over 156 weeks.

- Unit sales, households, visits, and spend data by product, store, and week
- Base Price and Actual Shelf Price, to determine a product’s discount, if any
- Promotional support details (e.g., sale tag, in-store display), if applicable for the given product/store/week
- Store information, including size and location, as well as a price tier designation (e.g., upscale vs. value)
- Product information, including UPC, size, and description

To identify outliers, it is suggested to look at

- The ratio of units vs. number of visits
- The ratio of visits vs. number of households
- Some items that may be out-of-stock or discontinued for a store

**Source:** https://www.dunnhumby.com/source-files/

In [2]:
from pyspark.sql import Row, SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("breakfast") \
    .getOrCreate()

In [4]:
product_data_folder = "dataset/products"
store_data_folder = "dataset/stores"
transaction_data_folder = "dataset/transactions"

### Perform ETL to Answer the Following Questions

1. What is the range of prices offered on products?
1. What is the impact on units/visit of promotions by geographies?
1. Which products would you lower the price to increase sales?

In [5]:
product_df = spark.read.option("header", True).csv(product_data_folder)

In [6]:
transaction_df = spark.read.option("header", True).csv(transaction_data_folder)

In [12]:
store_df = spark.read.option("header", True).csv(store_data_folder)

In [17]:
product_df.show(5)

+----------+--------------------+-------------+--------------------+--------------------+------------+
|       UPC|         DESCRIPTION| MANUFACTURER|            CATEGORY|        SUB_CATEGORY|PRODUCT_SIZE|
+----------+--------------------+-------------+--------------------+--------------------+------------+
|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|          BAG SNACKS|            PRETZELS|       15 OZ|
|1111009497|   PL PRETZEL STICKS|PRIVATE LABEL|          BAG SNACKS|            PRETZELS|       15 OZ|
|1111009507|   PL TWIST PRETZELS|PRIVATE LABEL|          BAG SNACKS|            PRETZELS|       15 OZ|
|1111035398|PL BL MINT ANTSPT...|PRIVATE LABEL|ORAL HYGIENE PROD...|MOUTHWASHES (ANTI...|      1.5 LT|
|1111038078|PL BL MINT ANTSPT...|PRIVATE LABEL|ORAL HYGIENE PROD...|MOUTHWASHES (ANTI...|      500 ML|
+----------+--------------------+-------------+--------------------+--------------------+------------+
only showing top 5 rows



In [16]:
transaction_df.show(10)

+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+
|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS|SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+
|    14-Jan-09|      367|1111009477|   13|    13| 13|18.07| 1.39|      1.57|      0|      0|       1|
|    14-Jan-09|      367|1111009497|   20|    18| 18| 27.8| 1.39|      1.39|      0|      0|       0|
|    14-Jan-09|      367|1111009507|   14|    14| 14|19.32| 1.38|      1.38|      0|      0|       0|
|    14-Jan-09|      367|1111035398|    4|     3|  3|   14|  3.5|      4.49|      0|      0|       1|
|    14-Jan-09|      367|1111038078|    3|     3|  3|  7.5|  2.5|       2.5|      0|      0|       0|
|    14-Jan-09|      367|1111038080|    2|     2|  2| 5.18| 2.59|      2.59|      0|      0|       0|
|    14-Jan-09|      367|1111085319|   14|    13| 13|26.32| 1.88|      1.88|      

In [47]:
store_df.show(5)

+--------+------------------+-----------------+-----------------------+--------+--------------+-----------------+-------------------+------------------+
|STORE_ID|        STORE_NAME|ADDRESS_CITY_NAME|ADDRESS_STATE_PROV_CODE|MSA_CODE|SEG_VALUE_NAME|PARKING_SPACE_QTY|SALES_AREA_SIZE_NUM|AVG_WEEKLY_BASKETS|
+--------+------------------+-----------------+-----------------------+--------+--------------+-----------------+-------------------+------------------+
|     389|        SILVERLAKE|         ERLANGER|                     KY|   17140|    MAINSTREAM|              408|              46073|             24767|
|    2277|ANDERSON TOWNE CTR|       CINCINNATI|                     OH|   17140|       UPSCALE|             null|              81958|             54053|
|    4259|     WARSAW AVENUE|       CINCINNATI|                     OH|   17140|         VALUE|             null|              48813|             31177|
|    6379|          KINGWOOD|         KINGWOOD|                     TX|   26420|  

In [53]:
store_df.filter(store_df["ADDRESS_STATE_PROV_CODE"] == "IN").show()

+--------+------------+-----------------+-----------------------+--------+--------------+-----------------+-------------------+------------------+
|STORE_ID|  STORE_NAME|ADDRESS_CITY_NAME|ADDRESS_STATE_PROV_CODE|MSA_CODE|SEG_VALUE_NAME|PARKING_SPACE_QTY|SALES_AREA_SIZE_NUM|AVG_WEEKLY_BASKETS|
+--------+------------+-----------------+-----------------------+--------+--------------+-----------------+-------------------+------------------+
|    4245|LAWRENCEBURG|     LAWRENCEBURG|                     IN|   17140|    MAINSTREAM|             null|              58563|             19275|
+--------+------------+-----------------+-----------------------+--------+--------------+-----------------+-------------------+------------------+



In [13]:
product_df.createOrReplaceTempView("products")
store_df.createOrReplaceTempView("stores")
transaction_df.createOrReplaceTempView("transactions")

In [8]:
df = spark.sql("""
    select
        *
        
    from transactions
    join products
    on 
        transactions.upc = products.upc
""")

In [9]:
df.show(1)

+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
|WEEK_END_DATE|STORE_NUM|       UPC|UNITS|VISITS|HHS|SPEND|PRICE|BASE_PRICE|FEATURE|DISPLAY|TPR_ONLY|       UPC|         DESCRIPTION| MANUFACTURER|  CATEGORY|SUB_CATEGORY|PRODUCT_SIZE|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
|    14-Jan-09|      367|1111009477|   13|    13| 13|18.07| 1.39|      1.57|      0|      0|       1|1111009477|PL MINI TWIST PRE...|PRIVATE LABEL|BAG SNACKS|    PRETZELS|       15 OZ|
+-------------+---------+----------+-----+------+---+-----+-----+----------+-------+-------+--------+----------+--------------------+-------------+----------+------------+------------+
only showing top 1 row



## What is the range of prices offered on products?


In [11]:
spark.sql("""
    select
        products.upc
        , min(price)
        , max(price)
        , description
        , category
        
    from transactions
    join products
    on 
        transactions.upc = products.upc
    group by
        1, 4, 5
""").show(3)

+----------+----------+----------+--------------------+----------+
|       upc|min(price)|max(price)|         description|  category|
+----------+----------+----------+--------------------+----------+
|1111009477|      0.89|      1.83|PL MINI TWIST PRE...|BAG SNACKS|
|1111009497|      0.86|      1.69|   PL PRETZEL STICKS|BAG SNACKS|
|1111009507|       0.8|      1.69|   PL TWIST PRETZELS|BAG SNACKS|
+----------+----------+----------+--------------------+----------+
only showing top 3 rows



In [10]:
spark.sql("""
    select
        upc
        , product_size
        , split(product_size, ' ')[0] as value
        , split(product_size, ' ')[1] as unit
        , case
            when split(product_size, ' ')[1] = 'OZ' then split(product_size, ' ')[0] * 100
            else
                split(product_size, ' ')[0] / 100
            end as is_oz
        
    from products
""").show(5)

+----------+------------+-----+----+------+
|       upc|product_size|value|unit| is_oz|
+----------+------------+-----+----+------+
|1111009477|       15 OZ|   15|  OZ|1500.0|
|1111009497|       15 OZ|   15|  OZ|1500.0|
|1111009507|       15 OZ|   15|  OZ|1500.0|
|1111035398|      1.5 LT|  1.5|  LT| 0.015|
|1111038078|      500 ML|  500|  ML|   5.0|
+----------+------------+-----+----+------+
only showing top 5 rows



## What is the impact on units/visit of promotions by geographies?

In [38]:
spark.sql("""
    select
        -- transactions.upc
        -- , transactions.STORE_NUM
        -- , price
        -- , base_price
        case 
            when (base_price - price) > 0  then 1 
            else 
                0
            end as is_sale
        -- , units
        -- , visits
        , units/visits as buy_per_p
        
        , ADDRESS_STATE_PROV_CODE
        
        
    from transactions
    join stores
    on 
        transactions.STORE_NUM = stores.STORE_ID
    where (base_price - price) > 0

    
""").show(5)

+-------+------------------+-----------------------+
|is_sale|         buy_per_p|ADDRESS_STATE_PROV_CODE|
+-------+------------------+-----------------------+
|      1|               1.0|                     KY|
|      1|1.3333333333333333|                     KY|
|      1|              1.25|                     KY|
|      1|1.4186046511627908|                     KY|
|      1|1.3846153846153846|                     KY|
+-------+------------------+-----------------------+
only showing top 5 rows



In [41]:
spark.sql("""
    select        
           
        AVG(units/visits) as buy_per_p        
        , ADDRESS_STATE_PROV_CODE        
        
    from transactions
    join stores
    on 
        transactions.STORE_NUM = stores.STORE_ID
    where (base_price - price) > 0
    group by
        ADDRESS_STATE_PROV_CODE

    
""").show(5)

+------------------+-----------------------+
|         buy_per_p|ADDRESS_STATE_PROV_CODE|
+------------------+-----------------------+
|1.1834336090000266|                     KY|
|1.1928337487593017|                     IN|
|1.1898599127461706|                     OH|
|1.1523177900229677|                     TX|
+------------------+-----------------------+



### from result above the sale item (promotion) in each STATE_PROV the best impact of promotion is "IN"  
#### Indiana state have more impact with promotion 

## Which products would you lower the price to increase sales?

In [59]:
spark.sql("""
    select
       
        CATEGORY
        -- , sub_CATEGORY
        , AVG(units/visits) as buy_per_p
        -- , SPEND
        
    from transactions
    join products
    on 
        transactions.upc = products.upc
    where (base_price - price) > 0
    group by
        CATEGORY

    
""").show(15)

+--------------------+------------------+
|            CATEGORY|         buy_per_p|
+--------------------+------------------+
|         COLD CEREAL| 1.168127166132834|
|        FROZEN PIZZA|1.1645997816954927|
|ORAL HYGIENE PROD...| 1.121139647543165|
|          BAG SNACKS|1.2368985083204926|
+--------------------+------------------+



### from result above, lower the price of "BAG SNACKS" can increase average unit sales per visit  
#### a person buy 1.237 unit per visit